# Core Tutorial

This script will introduce you to the basics of time series handling with pynapple.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pynapple as nap

## Time series object

Let's create a Tsd object with fake data. In this case, every time point is 1 second apart. A Tsd object is a wrapper of pandas series.

In [2]:
tsd = nap.Tsd(t = np.arange(100), d = np.random.rand(100), time_units = 's')

print(tsd)

Time (s)
0.0     0.350383
1.0     0.112004
2.0     0.559038
3.0     0.883592
4.0     0.783561
          ...   
95.0    0.989078
96.0    0.240785
97.0    0.127436
98.0    0.936579
99.0    0.575566
Length: 100, dtype: float64


It is possible to switch between seconds, milliseconds and microseconds. Note that when using *as_units*, the returned object is a simple pandas series.

In [3]:
print(tsd.as_units('ms'))
print(tsd.as_units('us'))

Time (ms)
0.0        0.350383
1000.0     0.112004
2000.0     0.559038
3000.0     0.883592
4000.0     0.783561
             ...   
95000.0    0.989078
96000.0    0.240785
97000.0    0.127436
98000.0    0.936579
99000.0    0.575566
Length: 100, dtype: float64
Time (us)
0           0.350383
1000000     0.112004
2000000     0.559038
3000000     0.883592
4000000     0.783561
              ...   
95000000    0.989078
96000000    0.240785
97000000    0.127436
98000000    0.936579
99000000    0.575566
Length: 100, dtype: float64


If only timestamps are available, for example spike times, we can construct a Ts object which holds only times. In this case, we generate 10 random spike times between 0 and 100 ms.

In [4]:
ts = nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 'ms')

print(ts)

Time (s)
0.019642   NaN
0.023977   NaN
0.032920   NaN
0.047616   NaN
0.048708   NaN
0.076251   NaN
0.087913   NaN
0.092497   NaN
0.095161   NaN
0.099498   NaN
dtype: float64


If the time series contains multiple columns, we can use a TsdFrame.

In [5]:
tsdframe = nap.TsdFrame(t = np.arange(100), 
                        d = np.random.rand(100,3), 
                        time_units = 's', 
                        columns = ['a', 'b', 'c'])

print(tsdframe)

                 a         b         c
Time (s)                              
0.0       0.357438  0.647921  0.982669
1.0       0.442921  0.162486  0.750832
2.0       0.283657  0.602511  0.322299
3.0       0.069515  0.987253  0.396827
4.0       0.244561  0.943814  0.540749
...            ...       ...       ...
95.0      0.476016  0.777918  0.518801
96.0      0.927694  0.129702  0.389647
97.0      0.820231  0.992514  0.589118
98.0      0.570093  0.897525  0.099027
99.0      0.937407  0.545003  0.032018

[100 rows x 3 columns]


## Interval Sets object

The [IntervalSet](https://peyrachelab.github.io/pynapple/core.interval_set/) object stores multiple epochs with a common time units. It can then be used to restrict time series to this particular set of epochs.

In [6]:
epochs = nap.IntervalSet(start = [0, 10], end = [5, 15], time_units = 's')

new_tsd = tsd.restrict(epochs)

print(epochs)
print('\n')
print(new_tsd)

   start   end
0    0.0   5.0
1   10.0  15.0


Time (s)
0.0     0.350383
1.0     0.112004
2.0     0.559038
3.0     0.883592
4.0     0.783561
5.0     0.367693
10.0    0.679473
11.0    0.181329
12.0    0.116674
13.0    0.107616
14.0    0.449670
15.0    0.579260
dtype: float64


Multiple operations are available for IntervalSet. For example, IntervalSet can be merged. See the full documentation of the class [here](https://peyrachelab.github.io/pynapple/core.interval_set/#pynapple.core.interval_set.IntervalSet.intersect) for a list of all the functions that can be used to manipulate IntervalSets.

In [7]:
epoch1 = nap.IntervalSet(start=[0], end=[10]) # no time units passed. Default is us.
epoch2 = nap.IntervalSet(start=[5,30],end=[20,45])

epoch = epoch1.union(epoch2)
print(epoch1, '\n')
print(epoch2, '\n')
print(epoch)

   start   end
0    0.0  10.0 

   start   end
0    5.0  20.0
1   30.0  45.0 

   start   end
0    0.0  20.0
1   30.0  45.0


## TsGroup

Multiple time series with different time stamps (.i.e. a group of neurons with different spike times from one session) can be grouped with the TsGroup object. The TsGroup behaves like a dictionnary but it is also possible to slice with a list of indexes

In [8]:
my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 1000)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 2000)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 3000)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

print(tsgroup, '\n')
print(tsgroup[0], '\n') # dictionnary like indexing returns directly the Ts object
print(tsgroup[[0,2]]) # list like indexing

  Index    Freq. (Hz)
-------  ------------
      0            10
      1            20
      2            30 

Time (s)
0.003141    NaN
0.067542    NaN
0.152387    NaN
0.360369    NaN
0.452814    NaN
             ..
99.467991   NaN
99.611809   NaN
99.647528   NaN
99.819072   NaN
99.912819   NaN
Length: 1000, dtype: float64 

  Index    Freq. (Hz)
-------  ------------
      0            10
      2            30


Operations such as restrict can thus be directly applied to the TsGroup as well as other operations.

In [9]:
newtsgroup = tsgroup.restrict(epochs)

count = tsgroup.count(1, epochs, time_units='s') # Here counting the elements within bins of 1 seconds

print(count)

           0   1   2
Time (s)            
0.0        7  14  28
1.0       10  27  36
2.0       10  15  38
3.0       14  19  31
4.0       10  22  30
10.0       4  21  37
11.0       8  20  30
12.0       6  20  27
13.0      15  18  32
14.0       7  19  29


One advantage of grouping time series is that metainformation can be added about each elements. In this case, we add labels to each Ts object when instantiating the group and after. We can then use this label to split the group. See the documentation about [TsGroup](https://peyrachelab.github.io/pynapple/core.ts_group/) for all the ways to split TsGroup.

In [10]:
tsgroup = nap.TsGroup(my_ts, time_units = 's', label1=[0,1,0])
tsgroup.set_info(label2=['a', 'a', 'b'])

print(tsgroup, '\n')

newtsgroup= tsgroup.getby_category('label1')
print(newtsgroup[0], '\n')
print(newtsgroup[1])


  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0            10         0  a
      1            20         1  a
      2            30         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0            10         0  a
      2            30         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      1            20         1  a


## Time support

A key element of the manipulation of time series by pynapple is the inherent time support defined for Ts, Tsd, TsdFrame and TsGroup objects. The time support is defined as an IntervalSet that provides the time serie with a context. For example,, the restrict operation will update automatically the time support to the new time series. Ideally the time support should be defined for all time series when instantiating them. If no time series is given, the time support is inferred from the start and end of the time series. 

In this example, a TsGroup is instantiated with and without a time support. Notice how the frequency of each Ts element is changed when the time support is defined explicitely.

In [11]:
time_support = nap.IntervalSet(start = 0, end = 200, time_units = 's')

my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 20)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 30)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

tsgroup_with_time_support = nap.TsGroup(my_ts, time_support = time_support)

print(tsgroup, '\n')

print(tsgroup_with_time_support, '\n')

print(tsgroup_with_time_support.time_support) # acceding the time support

  Index    Freq. (Hz)
-------  ------------
      0          0.1
      1          0.2
      2          0.31 

  Index    Freq. (Hz)
-------  ------------
      0          0.05
      1          0.1
      2          0.15 

   start    end
0    0.0  200.0
